In [38]:
key_length = 60 #each key pressed for 60 ms, minimal 50ms
delay = 10 #each key delayed for 10 ms before next key

In [39]:
import pandas 

In [40]:
path_input = '.\input.csv'
path_output = '.\output.csv'
df_input = pandas.read_csv(path_input)


In [41]:
df_input.head()

,output_falg,Stratagem_name,Stratagem_code,class_name
0,0,Machine Gun (level 1),Down&Left&Down&Up&Right,Patriotic Administration Center Stratagem Codes
1,0,Anti-Material Rifle (level 2),Down&Left&Right&Up&Down,Patriotic Administration Center Stratagem Codes
2,0,Stalwart (level 2),Down&Left&Down&Up&Up&Left,Patriotic Administration Center Stratagem Codes
3,1,Expendable Anti-Tank (level 3),Down&Down&Left&Up&Right,Patriotic Administration Center Stratagem Codes
4,2,Recoiled Rifle (level 5),Down&Left&Right&Right&Left,Patriotic Administration Center Stratagem Codes


In [42]:
path_json = '.\mf17.layout.json'
import json
 
# Opening JSON file
f = open(path_json)
 
# returns JSON object as 
# a dictionary
data_json = json.load(f)
 
# Closing file
f.close()

In [43]:
print(data_json['macros'][0])

{+KC_LCTL}{10}{+KC_DOWN}{60}{-KC_DOWN}{10}{+KC_DOWN}{60}{-KC_DOWN}{10}{+KC_LEFT}{60}{-KC_LEFT}{10}{+KC_UP}{60}{-KC_UP}{10}{+KC_RGHT}{60}{-KC_RGHT}{-KC_LCTL}{10}


In [44]:
def raw_code_formater(raw_code):
    dict1 = {
        "Down":"{1}",
        "Up":"{2}",
        "Left":"{3}",
        "Right":"{4}"
    }
    formated_code = raw_code
    for item in dict1:
        formated_code = formated_code.replace(item,dict1[item])
    return(formated_code)
#testing
raw_code_formater(df_input['Stratagem_code'][1])


'{1}&{3}&{4}&{2}&{1}'

In [45]:
def convert_2_macro(raw_code,key_length,delay):
    dict_converter = {
        'string_down1' : ('{1}','{+KC_DOWN}'+'{'+str(key_length)+'}'+'{-KC_DOWN}'),
        'string_up2' : ('{2}','{+KC_UP}'+'{'+str(key_length)+'}'+'{-KC_UP}'),
        'string_left3' : ('{3}','{+KC_LEFT}'+'{'+str(key_length)+'}'+'{-KC_LEFT}'),
        'string_right4' : ('{4}','{+KC_RGHT}'+'{'+str(key_length)+'}'+'{-KC_RGHT}'),
        '&' : ('&','{'+str(delay)+'}')
    }
    macro = raw_code_formater(raw_code)
    for item in dict_converter:
        macro = macro.replace(dict_converter[item][0],dict_converter[item][1])
    return(macro)

In [46]:
counts_row = df_input['Stratagem_name'].count()
df_output = pandas.DataFrame()
macro_list =[]
for i in range(0,counts_row):
    raw_codes = df_input['Stratagem_code'][i]
    macro = convert_2_macro(raw_codes,key_length,delay)
    macro = '{+KC_LCTL}{10}'+macro+'{-KC_LCTL}{10}'
    macro_list.append(macro)
df_input['macro'] = macro_list

In [47]:
df_input.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   output_falg     51 non-null     int64 
 1   Stratagem_name  51 non-null     object
 2   Stratagem_code  51 non-null     object
 3   class_name      51 non-null     object
 4   macro           51 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.1+ KB


In [48]:
code_df = df_input.loc[df_input['output_falg'] > 0]
code_df.reset_index(inplace=True,drop=True)
for i in range (0,code_df['macro'].count()):
    data_json['macros'][i] = code_df['macro'][i]



In [49]:
data_json
with open(path_json, 'w') as f:
    json.dump(data_json, f)